In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), \
    'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    
print( '==============' )


TensorFlow Version: 1.2.0
Default GPU Device: /gpu:0


In [2]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    meta_graph_def = tf.saved_model.loader.load( sess, [ vgg_tag ], vgg_path )

    image_input = sess.graph.get_tensor_by_name( vgg_input_tensor_name )
    keep_prob = sess.graph.get_tensor_by_name( vgg_keep_prob_tensor_name )
    layer3_out = sess.graph.get_tensor_by_name( vgg_layer3_out_tensor_name )
    layer4_out = sess.graph.get_tensor_by_name( vgg_layer4_out_tensor_name )
    layer7_out = sess.graph.get_tensor_by_name( vgg_layer7_out_tensor_name )
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out
tests.test_load_vgg(load_vgg, tf)



Tests Passed


In [3]:

def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    # output = tf.layers.conv2d( vgg_layer7_out, num_classes, 1, 1 )

    output = tf.layers.conv2d_transpose( vgg_layer7_out, 512, 4, 2, padding = 'same' )
    output = tf.add( output, vgg_layer4_out )
    output = tf.layers.conv2d_transpose( output, 256, 4, 2, padding = 'same' )
    output = tf.add( output, vgg_layer3_out )

    output = tf.layers.conv2d_transpose( output, num_classes, 16, 8, padding = 'same' )
    return output
tests.test_layers(layers)



Tests Passed


In [4]:

def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    logits = tf.reshape( nn_last_layer, ( -1, num_classes ) )
    labels = tf.reshape( correct_label, ( -1, num_classes ) )
    cross_entropy_loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits( \
        logits = logits, labels = labels ) )
    optimizer = tf.train.AdamOptimizer( learning_rate = learning_rate )
    train_op = optimizer.minimize( cross_entropy_loss )
    
    correct_pred = tf.equal( tf.argmax( logits, 1 ), tf.argmax( labels, 1 ) )
    accuracy_op = tf.reduce_mean( tf.cast( correct_pred, tf.float32 ) )

    return logits, train_op, cross_entropy_loss, accuracy_op
#tests.test_optimize(optimize)



In [5]:

def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate, accuracy_op):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    print( 'Training ...')
    print()
    for epoch in range( epochs ) :
        print( 'Epoch {} ...'.format( epoch + 1 ) )

        for batch_x, batch_y in get_batches_fn( batch_size ) :
            sess.run( train_op, feed_dict = {
                input_image: batch_x, correct_label: batch_y, keep_prob: 0.5, learning_rate: 0.001
                } )

        loss = sess.run( cross_entropy_loss, feed_dict = {
            input_image: batch_x, correct_label: batch_y, keep_prob: 1.
            } )

        print( 'Loss = {:.3f}'.format( loss ) )
        
        # Evaluate accuracy
        total_accuracy = 0.0
        num_samples = 0
        
        for batch_x, batch_y in get_batches_fn( batch_size ) :
            accuracy = sess.run( accuracy_op, feed_dict = {
                input_image: batch_x, correct_label: batch_y, keep_prob: 1.
                } )
            total_accuracy += accuracy * len( batch_x )
            num_samples += len( batch_x )

        print( 'Accuracy = {:.3f}'.format( total_accuracy / num_samples ) )
        print()

print( 'Done.' )


Done.


In [7]:
import numpy as np
import time

num_classes = 2
image_shape = (160, 576)
data_dir = './data'
runs_dir = './runs'

with tf.Session( graph = tf.Graph() ) as sess:
    # Path to vgg model
    vgg_path = os.path.join(data_dir, 'vgg')
    # Create function to get batches
    get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

    # TODO: Build NN using load_vgg, layers, and optimize function
    input_image, keep_prob, layer3_out, layer4_out, layer7_out = \
        load_vgg( sess, os.path.join( data_dir, 'vgg' ) )

    # ===== test ======
    # conv1_1/filter:0 ===> [3, 3, 3, 64]
#     vs = tf.get_collection( tf.GraphKeys.TRAINABLE_VARIABLES )
#     v0 = vs[0].eval()
#     print( 'trainable:', len(vs), vs[0].name, v0.shape )
    
#     v0 = tf.get_variable( 'conv1_1/filter:0' ).eval()
    
#     for v in vs:
#         print( v.name, '===>', v.get_shape().as_list() )
    
    # ===== test end =======

#     layer7_out = tf.stop_gradient( layer7_out )
    
    output = layers( layer3_out, layer4_out, layer7_out, num_classes )

    correct_label = tf.placeholder( tf.int32, shape = ( None, *image_shape, num_classes ) )
    learning_rate = tf.placeholder( tf.float32 )
    logits, train_op, cross_entropy_loss, accuracy_op = optimize( output, correct_label, learning_rate, num_classes )

    # TODO: Train NN using the train_nn function
    epochs = 30
    batch_size = 20

    sess.run( tf.global_variables_initializer() )
    
    time_start = time.time()
    
    print()
    train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
         correct_label, keep_prob, learning_rate, accuracy_op)
    print()
    
    print( 'Train end with time {:.3f}s'.format( time.time() - time_start ) )
    
#     vs_1 = tf.get_collection( tf.GraphKeys.TRAINABLE_VARIABLES )
#     v0_1 = vs_1[0].eval()
#     print( 'trainable_1:', len(vs_1), vs_1[0].name, v0_1.shape )
    
#     v0_1 = tf.get_variable( 'conv1_1/filter:0' ).eval()
#     print( '===var check:', np.all( v0 == v0_1 ) )

    # TODO: Save inference data using helper.save_inference_samples
    #  helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
    helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

    # OPTIONAL: Apply the trained model to a video

INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'

Training ...

Epoch 1 ...
Loss = 14.307
Accuracy = 0.805

Epoch 2 ...
Loss = 20.161
Accuracy = 0.707

Epoch 3 ...
Loss = 7.776
Accuracy = 0.831

Epoch 4 ...
Loss = 3.604
Accuracy = 0.824

Epoch 5 ...
Loss = 2.858
Accuracy = 0.804

Epoch 6 ...
Loss = 1.632
Accuracy = 0.848

Epoch 7 ...
Loss = 1.074
Accuracy = 0.831

Epoch 8 ...
Loss = 0.770
Accuracy = 0.883

Epoch 9 ...
Loss = 0.643
Accuracy = 0.871

Epoch 10 ...
Loss = 0.514
Accuracy = 0.895

Epoch 11 ...
Loss = 0.404
Accuracy = 0.888

Epoch 12 ...
Loss = 0.691
Accuracy = 0.883

Epoch 13 ...
Loss = 0.389
Accuracy = 0.897

Epoch 14 ...
Loss = 0.283
Accuracy = 0.906

Epoch 15 ...
Loss = 0.393
Accuracy = 0.892

Epoch 16 ...
Loss = 0.330
Accuracy = 0.912

Epoch 17 ...
Loss = 0.224
Accuracy = 0.915

Epoch 18 ...
Loss = 0.240
Accuracy = 0.909

Epoch 19 ...
Loss = 0.334
Accuracy = 0.907

Epoch 20 ...
Loss = 0.227
Accuracy = 0.915

Epoch 21 ...
Loss = 0.202
Accuracy =